In [3]:
#include <lodepng_config_cling.h>
#include <vtkm_config_cling.h>
#include <iostream>
#include <string>
#include <VTKmXeusRender.h>
#include <adios2.h>
#include <xwidgets/slider.h>

#pragma cling load("adios2")

input_line_11:7:10: fatal error: 'xwidgets/slider.h' file not found
#include <xwidgets/slider.h>
         ^~~~~~~~~~~~~~~~~~~


Interpreter Error: 

In [2]:
#include <vtkm/cont/testing/MakeTestDataSet.h>
std::unique_ptr<VTKmXeusRender> renderer;
std::unique_ptr<adios2::ADIOS> adios;
adios2::IO bpIO;
adios2::Engine bpReader;
xw::slider slider;



input_line_10:6:1: error: use of undeclared identifier 'xw'
xw::slider slider;
^


Interpreter Error: 

In [ ]:

auto openBP(std::string fn)
{
    

    adios = std::make_unique<adios2::ADIOS>(adios2::DebugON);
    bpIO = adios->DeclareIO("ReadBP");        

    std::stringstream filename;
    filename << "/home/ybk/Projects/raytracingtherestofyourlife/build/debug/";
    filename << fn;
    /** Engine derived class, spawned to start IO operations */
    bpReader = bpIO.Open(filename.str(), adios2::Mode::Read);
}

In [ ]:
auto openVariable(std::string vn)
{
    /** Write variable for buffering */
    adios2::Variable<float> bpFloats =
    bpIO.InquireVariable<float>(vn);    
    return std::make_tuple(bpReader, bpFloats);
}


In [ ]:

auto open(std::string fn, std::string vn)
{
    openBP(fn);
    return openVariable(vn);
}


In [ ]:
void queryVariables()
{
    const auto variables = bpIO.AvailableVariables();
    std::cout << variables.size() << std::endl;
    for (const auto variablePair : variables)
    {
         std::cout << "Name: " << variablePair.first;
//           for (const auto &parameter : variablePair.second)
//           {
              
//               std::cout << "\t" << parameter.first << ": " << parameter.second
//                           << "\n";
//           }
        std::cout << std::endl;
    }
}


In [ ]:
void display(std::tuple<adios2::Engine,adios2::Variable<float>> tup, int nx, int ny)
{
    adios2::Engine bpReader;
    adios2::Variable<float> bpFloats;
    
    std::tie(bpReader, bpFloats) = tup;

    if (bpFloats)
    {
        
        std::vector<float> depthBuffer;
        std::vector<vtkm::Vec<vtkm::Float32,4>> useful;
        useful.resize(nx*ny);
        bpReader.Get<float>(bpFloats, depthBuffer, adios2::Mode::Sync);
  
         for (int i=0; i<useful.size(); i++)
        {
            //std::cout << x << ", ";
            auto &uf = useful[i];
            uf[0] = depthBuffer[i*4];
            uf[1] = depthBuffer[i*4+1];
            uf[2] = depthBuffer[i*4+2];
            uf[3] = 0;
            
        }
        vtkm::cont::ArrayHandle arr = vtkm::cont::make_ArrayHandle(useful);
        renderer->Display(arr, nx,ny);
    }
}

In [ ]:
[](){auto try_and_err = []() {
    display(slider);
    renderer = std::make_unique<VTKmXeusRender>();
    auto tup = open("normals", "0.6444-5.4454");
    display(tup, 128, 128);
    
};try_and_err();}();

In [ ]:
[](){auto try_and_err = []() {
    //std::string bpname("normals");
    std::string bpname("albedo");
    //std::string bpname("direct");
    renderer = std::make_unique<VTKmXeusRender>();
    openBP(bpname);
    
    const auto variables = bpIO.AvailableVariables();
    std::cout << variables.size() << std::endl;
    for (const auto variablePair : variables)
    {
        auto tup = openVariable(variablePair.first);
        display(tup, 128, 128);
    }
    
};try_and_err();}();